In [1]:
import pandas as pd

print("Pandas version:", pd.__version__)

print("This script is used to test the integration of pandas in the project.")

Pandas version: 2.3.0
This script is used to test the integration of pandas in the project.
